# Logistic regression for a 2D classification problem

Imports...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

## Load and visualize data

In [ ]:
data = pd.read_csv("2d-classification-data.csv")
data.head(10)

In [ ]:
data.plot.scatter(x="x1", y="x2", c="y");

## Define and fit the model

In [ ]:
# Select model input variables
X = data[["x1", "x2"]].copy()
X

In [ ]:
# Define a model and fit it
# Note: sklearn.linear_model.LogisticRegression adds an intercept automatically.
model = LogisticRegression(penalty=None)
model.fit(X, data["y"])

In [ ]:
# Parameters
beta_1, beta_2 = model.coef_[0]
beta_0 = model.intercept_.item()
print(beta_0, beta_1, beta_2)

In [ ]:
# Alternative: statsmodels
model_sm = sm.Logit(data["y"], sm.add_constant(X))
results = model_sm.fit()
print(results.summary())

# beta_0, beta_1, beta_2 = results.params
# print(beta_0, beta_1, beta_2)

## Use model for Prediction

In [ ]:
# Prediction of training samples
data['prediction'] = model.predict(X)
data['probability'] = model.predict_proba(X)[:,1]

In [ ]:
# Alternative: statsmodels
data['prediction_sm'] = (model_sm.predict(results.params) > 0.5) * 1.0
data['probability_sm'] = model_sm.predict(results.params)

## Visualization

In [ ]:
# Predictions with decision boundary
data.plot.scatter(x="x1", y="x2", c="prediction");
x1 = np.array([-3, 3])
x2 = (- x1 * beta_1 - beta_0) / beta_2
plt.plot(x1, x2);

In [ ]:
# Decision boundary vs. ground truth
data.plot.scatter(x="x1", y="x2", c="y");
x1 = np.array([-3, 3])
x2 = (- x1 * beta_1 - beta_0) / beta_2
plt.plot(x1, x2);

In [ ]:
# Probabilities
data.plot.scatter(x="x1", y="x2", c="probability");
x1 = np.array([-3, 3])
x2 = (- x1 * beta_1 - beta_0) / beta_2
plt.plot(x1, x2);

In [ ]:
# data.to_csv("results.csv", index=False)

In [ ]:
# Bonus: Visualize learned decision probability (here: with statsmodels)
X, Y = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-4, 4, 100))
points = np.concatenate([X[..., None], Y[..., None]], axis=-1).reshape(-1, 2)
probs = model_sm.predict(results.params, sm.add_constant(points))

plt.scatter(X, Y, c=probs);
plt.colorbar();
plt.xlabel("$x_1$");
plt.ylabel("$x_2$");
plt.title("Probability $p(y = 1 | x)$");

from matplotlib import cm
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(X.reshape(100, 100), 
                       Y.reshape(100, 100), 
                       probs.reshape(100, 100), 
                       cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_xlabel("$x_1$");
ax.set_ylabel("$x_2$");
ax.set_zlabel("$p(y = 1 | x)$");
ax.view_init(30, 360-155);
ax.zaxis.labelpad=-2.5;
fig.tight_layout();